In [6]:
import itertools as it
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV

### Conjunto de dados

Primeiramente, vamos carregar as bases de dados e separar os rótulos que serão preditos.


In [14]:
# Características
df_sociais = pd.read_csv("data/trabalho5_dados_sociais_4.csv")
df_modulo1 = pd.read_csv("data/trabalho5_dados_modulo1_4.csv")
df_modulo2 = pd.read_csv("data/trabalho5_dados_ateh_modulo2_4.csv")
# Rótulos
y = LabelEncoder().fit(["Sim", "Não"]).transform(df_modulo2["aprovado"])
#df_sociais = df_sociais.drop(["id", "aprovado"], axis=1)
#df_modulo1 = df_modulo1.drop(["id", "aprovado"], axis=1)
#df_modulo2 = df_modulo2.drop(["id", "aprovado"], axis=1)

In [20]:
df_modulo1

,id,quesm1,quesm1r,forum1,forum2,forum3,forum4,ativcolm1,ativcolm1r,forum1r,aprovado
0,7,1,0,1,0,1,1,1,0,0,Sim
1,33,1,0,1,1,1,1,1,0,1,Sim
2,45,1,0,1,1,1,1,1,0,0,Sim
3,50,1,0,0,0,1,0,1,0,0,Sim
4,58,0,0,0,0,0,0,0,0,0,Não
...,...,...,...,...,...,...,...,...,...,...,...
995,10859,1,1,1,1,1,1,1,1,1,Sim
996,10872,0,1,1,0,0,0,1,0,1,Sim
997,10873,1,1,0,1,1,1,1,0,1,Sim
998,10876,1,1,1,1,1,1,1,0,1,Sim


A base de dados sociais é a única que possui variáveis multi-categóricas e ordinais não processadas. Vamos pré-processá-las para que possam ser usadas pelos classificadores.


In [8]:
# Ordens das categorias
sn_cat = ["Sim", "Não"]
sexo_cat = ["Feminino", "Masculino"]
escolaridade_cat = [
    "Ensino Médio Completo",
    "Ensino Superior Incompleto",
    "Ensino Superior Completo",
    "Pós-graduação",
]
materialdidatico_cat = ["Adequado", "Muito adequado"]
prazoatividades_cat = [
    "Pouquíssimo flexível",
    "Pouco flexível",
    "Flexível",
    "Muito flexível",
]
interacaopares_cat = ["Importante", "Muito importante"]
organizacaocurso_cat = ["Organizado", "Muito organizado"]
import_ajud_tutor_cat = ["Às vezes", "Sempre"]
autoavaliacao_cat = [
    "Não, não considero",
    "Sim, considero, porém, poderia estar me esforçando mais",
    "Sim, considero",
]
pp_cat = [
    "Discordo totalmente",
    "Discordo",
    "Nem discordo, nem concordo",
    "Concordo",
    "Concordo totalmente",
]

In [9]:
# Variáveis ordinais
cols_ord = [
    "escolaridade",
    "materialdidatico",
    "prazoatividades",
    "interacaopares",
    "import.ajud.tutor",
    "autoavaliacao.x",
] + [f"pp{n + 1:03}" for n in range(37)]
cats_ord = [
    escolaridade_cat,
    materialdidatico_cat,
    prazoatividades_cat,
    interacaopares_cat,
    import_ajud_tutor_cat,
    autoavaliacao_cat,
] + [pp_cat] * 37

# Variáveis nominais
cols_nom = list(set(df_sociais.select_dtypes(object).columns) - set(cols_ord))

# Variáveis numéricas
cols_num = ["idade", "tempodeservico"]

In [10]:
def search_results(X, y, column_combinations, results_path, cache=True):
    try:
        if cache:
            with open(results_path, "rb") as file:
                search_results = pickle.load(file)
        else:
            raise FileNotFoundError()
    except FileNotFoundError:
        search_results = []
        for cols in column_combinations:
            cols_ord_ = list(set(cols_ord) & set(cols))
            cats_ord_ = [cats_ord[cols_ord.index(x)] for x in cols_ord_]
            cols_nom_ = list(set(cols_nom) & set(cols))
            cols_num_ = list(set(cols_num) & set(cols))

            transformers = []
            if cols_ord_:
                transformers.append((OrdinalEncoder(categories=cats_ord_), cols_ord_))
            if cols_nom_:
                transformers.append((OneHotEncoder(), cols_nom_))
            if cols_num_:
                transformers.append((StandardScaler(), cols_num_))

            steps = []
            if transformers:
                steps.append(("transformer", make_column_transformer(*transformers)))
            steps.append(("estimator", LogisticRegression())),  # dummy
            pipe = Pipeline(steps)

            search = GridSearchCV(
                pipe,
                param_grid=[
                    {
                        "estimator": [LogisticRegression()],
                    },
                    {
                        "estimator": [GradientBoostingClassifier()],
                    },
                    {
                        "estimator": [RandomForestClassifier()],
                    },
                ],
                n_jobs=-1,
            )
            _ = search.fit(X[cols], y)

            result = pd.DataFrame(search.cv_results_)
            result["columns"] = [cols] * len(result)
            search_results.append(result)

        search_results = pd.concat(search_results).sort_values(
            by="mean_test_score", ascending=False
        )
        with open(results_path, "wb") as file:
            pickle.dump(search_results, file)

    return search_results

### Dados Sociais


In [11]:
column_combinations = [
    list(cols)
    for n_cols in range(1, len(df_sociais.columns) + 1)
    for cols in it.islice(it.combinations(df_sociais.columns, n_cols), 2)
]
results_sociais = search_results(
    X=df_sociais,
    y=y,
    column_combinations=column_combinations,
    results_path="data/search_results_1.pkl",
    cache=False,
)
results_sociais.head()

C:\Users\vinic\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\vinic\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,columns
0,0.023737,0.002221,0.006982,1.544649e-03,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.575,0.570,0.57,0.569,0.003742,1,"[idade, sexo, escolaridade, estadocivil, tempo..."
0,0.013688,0.006172,0.003608,5.835306e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,[idade]
0,0.017353,0.004398,0.004588,4.883442e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,"[idade, sexo]"
0,0.016556,0.000798,0.004987,1.040336e-06,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,"[idade, sexo, estadocivil]"
0,0.017753,0.002631,0.005984,3.693565e-07,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,"[idade, sexo, escolaridade]"


In [155]:
results_sociais.describe()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
count,465.000000,465.000000,465.000000,4.650000e+02,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000,465.000000
mean,0.147937,0.006166,0.012125,1.450947e-03,0.536677,0.479140,0.490280,0.506516,0.528538,0.508230,0.031404,1.978495
std,0.083695,0.005119,0.004491,1.284821e-03,0.026427,0.040995,0.036504,0.028118,0.017878,0.017663,0.011985,0.827576
min,0.004801,0.000400,0.001800,9.536743e-08,0.470000,0.410000,0.415000,0.430000,0.475000,0.471000,0.002000,1.000000
25%,0.056613,0.002136,0.009002,4.900961e-04,0.525000,0.450000,0.460000,0.485000,0.520000,0.498000,0.025179,1.000000
50%,0.168237,0.004119,0.012603,1.020039e-03,0.535000,0.465000,0.490000,0.510000,0.525000,0.510000,0.031718,2.000000
75%,0.231052,0.009917,0.015204,1.959927e-03,0.555000,0.510000,0.515000,0.525000,0.535000,0.515000,0.039623,3.000000
max,0.296267,0.036490,0.024602,9.606115e-03,0.605000,0.580000,0.575000,0.570000,0.575000,0.569000,0.061887,3.000000


### Dados dos módulos

In [152]:
df_modulo = pd.concat([df_modulo1, df_modulo2], axis=1)

column_combinations = [
    list(cols)
    for n_cols in range(1, len(df_modulo.columns) + 1)
    for cols in it.islice(it.combinations(df_modulo.columns, n_cols), 2)
]
results_modulos = search_results(
    X=df_modulo,
    y=y,
    column_combinations=column_combinations,
    results_path="data/search_results_1.pkl",
    cache=False,
)
results_modulos.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,columns
0,0.008221,0.000398,0.001396,4.847232e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.675,0.715,0.740,0.730,0.705,0.713,0.022494,1,"[quesm1, quesm1r, forum1, forum2, forum3, foru..."
0,0.007610,0.000589,0.001609,5.820859e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.675,0.715,0.740,0.720,0.715,0.713,0.021119,1,"[quesm1, quesm1r, forum1, forum2, forum3, foru..."
0,0.007499,0.000449,0.001400,4.899792e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.640,0.735,0.740,0.715,0.720,0.710,0.036194,1,"[quesm1, quesm1r, forum1, forum2, forum3, foru..."
1,0.111025,0.002098,0.002201,4.003764e-04,GradientBoostingClassifier(),{'estimator': GradientBoostingClassifier()},0.675,0.750,0.685,0.745,0.690,0.709,0.031843,1,"[quesm1, quesm1r, forum1, forum2, forum3, foru..."
1,0.120027,0.002608,0.002000,4.156970e-07,GradientBoostingClassifier(),{'estimator': GradientBoostingClassifier()},0.675,0.750,0.685,0.745,0.690,0.709,0.031843,1,"[quesm1, quesm1r, forum1, forum2, forum3, foru..."


In [156]:
results_modulos.describe()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
count,159.000000,1.590000e+02,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000
mean,0.080293,2.761748e-03,0.004076,0.000564,0.642107,0.688270,0.675535,0.686761,0.669151,0.672365,0.023981,1.918239
std,0.057360,2.477731e-03,0.003544,0.000404,0.043811,0.053796,0.047409,0.046307,0.042252,0.042738,0.008595,0.834149
min,0.003201,1.168008e-07,0.001000,0.000000,0.520000,0.575000,0.565000,0.585000,0.550000,0.573000,0.002000,1.000000
25%,0.009002,8.001328e-04,0.001497,0.000400,0.632500,0.650000,0.650000,0.655000,0.650000,0.656500,0.016793,1.000000
50%,0.099223,1.999950e-03,0.001803,0.000490,0.655000,0.705000,0.685000,0.695000,0.690000,0.692000,0.025962,2.000000
75%,0.130429,4.297512e-03,0.008202,0.000748,0.675000,0.730000,0.700000,0.720000,0.700000,0.704000,0.031401,3.000000
max,0.162836,1.303535e-02,0.012003,0.001624,0.705000,0.750000,0.765000,0.760000,0.720000,0.713000,0.040125,3.000000


### Todos os dados

In [153]:
df_all = pd.concat([df_sociais, df_modulo1, df_modulo2], axis=1)

column_combinations = [
    list(cols)
    for n_cols in range(1, len(df_all.columns) + 1)
    for cols in it.islice(it.combinations(df_all.columns, n_cols), 2)
]
results_all = search_results(
    X=df_all,
    y=y,
    column_combinations=column_combinations,
    results_path="data/search_results_1.pkl",
    cache=False,
)
results_all.head()

C:\Users\Work\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Work\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,columns
0,0.016204,0.001327,0.004201,4.003525e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.575,0.570,0.57,0.569,0.003742,1,"[idade, sexo, escolaridade, estadocivil, tempo..."
0,0.005601,0.000490,0.001801,3.997088e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,[idade]
0,0.015203,0.001600,0.004001,1.907349e-07,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,"[idade, sexo, escolaridade, estadocivil]"
0,0.011803,0.000749,0.003001,2.861023e-07,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,"[idade, sexo, estadocivil]"
0,0.011203,0.000749,0.003601,4.902128e-04,LogisticRegression(),{'estimator': LogisticRegression()},0.565,0.565,0.570,0.565,0.57,0.567,0.002449,1,"[idade, sexo, escolaridade]"


In [154]:
results_all.describe()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
count,627.000000,627.000000,627.000000,6.270000e+02,627.000000,627.000000,627.000000,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.161358,0.007874,0.013650,2.036335e-03,0.537041,0.476834,0.485598,0.507632,0.527464,0.506914,0.031957,1.979266
std,0.088646,0.006286,0.004773,2.236802e-03,0.023796,0.038396,0.035999,0.029031,0.015918,0.016701,0.011113,0.825638
min,0.005601,0.000400,0.001400,1.168008e-07,0.470000,0.410000,0.405000,0.430000,0.480000,0.470000,0.002000,1.000000
25%,0.059213,0.003213,0.011003,7.433807e-04,0.525000,0.450000,0.460000,0.485000,0.520000,0.496000,0.026721,1.000000
50%,0.190843,0.006257,0.013803,1.356853e-03,0.535000,0.460000,0.485000,0.515000,0.525000,0.511000,0.031401,2.000000
75%,0.245055,0.011703,0.017470,2.512795e-03,0.555000,0.510000,0.510000,0.535000,0.535000,0.514000,0.039774,3.000000
max,0.377084,0.085963,0.036008,2.350830e-02,0.600000,0.580000,0.575000,0.570000,0.575000,0.569000,0.062338,3.000000
